# 🌳 1D to 2D Classification using Decision Trees

This notebook demonstrates how to classify data by learning the mapping from 1D measurements (e.g., chord lengths) to 2D shape categories using decision tree algorithms. Such tasks arise in stereology and materials analysis where observed features are lower-dimensional projections of higher-dimensional structures.


## Inferring Sphere Radius from Chord Length Distributions

This script simulates 2D cuts through randomly packed spheres and collects the resulting chord lengths. These are converted into binned histograms (feature vectors) used to identify the original sphere radius via pattern matching.


In [119]:
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import pickle

def generate_spheres(num_spheres, radius):
    positions = []
    for _ in range(num_spheres):
        while True:
            x = np.random.uniform(radius, 1 - radius)
            y = np.random.uniform(radius, 1 - radius)
            if all(np.sqrt((x - px) ** 2 + (y - py) ** 2) >= 2 * radius for px, py in positions):
                positions.append((x, y))
                break
    return positions

def chord_length(radius, d_line):
    return 2 * np.sqrt(radius**2 - d_line**2)

def distance_from_line(a, b, x, y):
    return abs(a * x - y + b) / np.sqrt(a**2 + 1)

def collect_chords_for_cuts(positions, radius, num_cuts, y_range=(0, 1)):
    chord_lengths_per_cut = []
    # Spacing for horizontal cuts
    y_cut_values = np.linspace(y_range[0], y_range[1], num_cuts) 
    # Collect chord lengths for each cut
    for y_cut in y_cut_values:
        cut_chords = []
        for (x, y) in positions:
            # Calculate the perpendicular distance from the sphere center to the cut
            d_line = abs(y - y_cut)  # This is simply the vertical distance since it's a horizontal line

            # Check if the line intersects the sphere (distance must be less than the radius)
            if d_line < radius:
                length = chord_length(radius, d_line)
                cut_chords.append(length)      
        # Sort the chords for the current cut
        chord_lengths_per_cut.append(sorted(cut_chords))
    
    return chord_lengths_per_cut


def discretize_chord_lengths(chord_lengths, num_bins, range_min = 0, range_max=0.16):
    bin_edges = np.linspace(range_min, range_max, num_bins + 1)
    
    # Initialize a list to store the count of chord lengths in each bin
    bin_counts = np.zeros(num_bins, dtype=int)

    # For each chord length, find which bin it belongs to and increment the count for that bin
    for chord in chord_lengths:
        for i in range(num_bins):
            if bin_edges[i] < chord <= bin_edges[i + 1]:
                bin_counts[i] += 1
                break

    return bin_counts
    

def distribution_chord_lengths(chord_lengths_per_cut, num_bins):
    discretized_vectors = []
    
    for chord_lengths in chord_lengths_per_cut:
        if chord_lengths:  # If it's not empty, discretize
            discretized_vector = discretize_chord_lengths(chord_lengths, num_bins)
        #else:  # If it's empty, create a zeroed vector of the same length as the bins
        #    discretized_vector = np.zeros(num_bins, dtype=int)
        #
            discretized_vectors.append(discretized_vector)
    
    return discretized_vectors


def flatten_discretized_vectors(discretized_vectors):
    # Flatten each discretized vector into a 1D feature vector
    return np.array([vec.flatten() for vec in discretized_vectors])

def create_feature_radius_pairs(distributions, true_radii):
    # Ensure the number of distributions matches the number of true radii
    if len(distributions) != len(true_radii):
        raise ValueError("The number of distributions must match the number of true radii.")
    
    # Flatten the distributions
    flattened_distributions = [flatten_discretized_vectors(distribution) for distribution in distributions]
    
    # Pair each flattened vector with its corresponding true radius
    feature_radius_pairs = []
    for i, flattened_distribution in enumerate(flattened_distributions):
        for feature_vector in flattened_distribution:
            feature_radius_pairs.append((feature_vector, true_radii[i]))
    
    return feature_radius_pairs

def compute_vector_frequencies(distribution):
    vector_counts = defaultdict(int)

    for vector in distribution:
        vector_tuple = tuple(vector)  # Convert numpy array to tuple for hashing
        vector_counts[vector_tuple] += 1

    return vector_counts


def discretize_vector(vector, num_bins=10, range_min=0, range_max=0.16):
    # Discretize the observed vector using the same bins as the training data
    bin_edges = np.linspace(range_min, range_max, num_bins + 1)
    discretized_vector = np.zeros(num_bins, dtype=int)

    for val in vector:
        for i in range(num_bins):
            if bin_edges[i] < val <= bin_edges[i + 1]:
                discretized_vector[i] += 1
                break

    return tuple(discretized_vector)  # Convert to tuple for comparison in dictionary

# Function to find the best match across multiple dictionaries
def find_best_match_in_dictionaries(observed_vector, dictionaries, num_bins=10, range_min=0, range_max=0.16):
    # Discretize the observed vector
    discretized_observed = discretize_vector(observed_vector, num_bins, range_min, range_max)
    #discretized_observed = (np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0))
    best_match = None
    best_frequency = 0
    best_dict_name = None

    # Loop through each dictionary and check for matches
    for dict_name, vector_frequencies in dictionaries.items():
        # Find matches in the current dictionary
        matching_frequencies = {vec: count for vec, count in vector_frequencies.items() if vec == discretized_observed}
        
        if matching_frequencies:
            # Find the highest frequency in this dictionary
            max_frequency_in_dict = max(matching_frequencies.values())
            
            if max_frequency_in_dict > best_frequency:
                best_frequency = max_frequency_in_dict
                best_match = discretized_observed
                best_dict_name = dict_name

    return best_match, best_frequency, best_dict_name

## Simulating Chord Distributions for Different Sphere Densities

We simulate 2D cross-sections through sets of non-overlapping spheres (with fixed radius) at four different densities. For each case, we compute chord lengths from horizontal cuts and discretize them into histograms for further analysis.


In [120]:
# Parameters
num_cuts = 100  # Number of horizontal cuts
num_bins = 10 #feature vectors

radius = 0.08  # Radius of each circle

positions = generate_spheres(17, radius)
chord_lengths_set_17 = collect_chords_for_cuts(positions, radius, num_cuts)
distribution_17 = distribution_chord_lengths(chord_lengths_set_17, num_bins) #discretised vectors


positions = generate_spheres(15, radius)
chord_lengths_set_15 = collect_chords_for_cuts(positions, radius, num_cuts)
distribution_15 = distribution_chord_lengths(chord_lengths_set_15, num_bins) #discretised vectors


positions = generate_spheres(13, radius)
chord_lengths_set_13 = collect_chords_for_cuts(positions, radius, num_cuts)
distribution_13 = distribution_chord_lengths(chord_lengths_set_13, num_bins) #discretised vectors


positions = generate_spheres(11, radius)
chord_lengths_set_11 = collect_chords_for_cuts(positions, radius, num_cuts)
distribution_11 = distribution_chord_lengths(chord_lengths_set_11, num_bins) #discretised vectors

In [121]:


def prepare_numpy_dataset(num_spheres_list, radius, num_cuts, num_bins):
    dataset = []

    for num_spheres in num_spheres_list:
        positions = generate_spheres(num_spheres, radius)
        chord_lengths_set = collect_chords_for_cuts(positions, radius, num_cuts)
        discretized_vectors = distribution_chord_lengths(chord_lengths_set, num_bins)

        for vector in discretized_vectors:
            dataset.append(list(vector) + [num_spheres])  # додаємо мітку наприкінці

    data = np.array(dataset, dtype=np.float32)  # можна також int, якщо потрібно
    X = data[:, :-1]  # всі ознаки
    Y = data[:, -1]   # всі мітки (кількість сфер)

    return X, Y


In [122]:
def train_test_split(X, Y, test_size=0.2, shuffle=True):
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X = X[indices]
        Y = Y[indices]

    split_point = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_point], X[split_point:]
    Y_train, Y_test = Y[:split_point], Y[split_point:]

    return X_train, X_test, Y_train, Y_test

In [123]:
class RandomForest:
    def __init__(self,x_data,y_data,numb_of_tree,depth):
        self.x_data=x_data
        self.y_data=y_data
        self.numb_of_tree=numb_of_tree
        self.depth=depth

    def find_best_split(self,X, y, feature_indices):
        best_gini = float('inf')
        best_feature = None
        best_thresh = None

        for feat in feature_indices:
            thresholds = np.unique(X[:, feat])
            for t in thresholds:
                left_mask = X[:, feat] < t
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                gini = self.gini_split(y[left_mask], y[right_mask])
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feat
                    best_thresh = t

        return best_feature, best_thresh
    def gini_split(self,y_left, y_right):
        def gini(y):
            classes, counts = np.unique(y, return_counts=True)
            probs = counts / counts.sum()
            return 1 - np.sum(probs ** 2)
        n = len(y_left) + len(y_right)
        return (len(y_left) / n) * gini(y_left) + (len(y_right) / n) * gini(y_right)
    def predict_tree(self,x, node):
        if node["is_leaf"]:
            return node["prediction"]
        if x[node["feature"]] < node["threshold"]:
            return self.predict_tree(x, node["left"])
        else:
            return self.predict_tree(x, node["right"])
    def random_feature(self,numb_of_feat,size):
        arr1=[x for x in range(numb_of_feat)]
        unique_numbers = np.unique(arr1)
        result=np.random.choice(unique_numbers,size=size,replace=False)
        return result
    def most_common(self,y):
        values, counts = np.unique(y, return_counts=True)  # Знаходить унікальні значення та їх кількість
        most_common_value = values[np.argmax(counts)]      # Знаходить значення з найбільшою кількістю
        return most_common_value
    def build_tree(self,X, y, feature_indices, depth=0):
        # 1. Перевірка: якщо всі мітки однакові → зупиняємось
        if np.all(y == y[0]):
            return {'is_leaf': True, 'prediction': y[0]}
    
        if depth == self.depth:
            majority_class = self.most_common(y)
            return {'is_leaf': True, 'prediction': majority_class}

        # 3. Шукаємо найкраще розбиття
        best_feature, best_thresh = self.find_best_split(X, y, feature_indices)
        if best_feature is None or best_thresh is None:
            majority_class = self.most_common(y)  # Базуємося на більшості у поточному наборі
            return {'is_leaf': True, 'prediction': majority_class}
        # 4. Розділяємо дані
        left_mask = X[:, best_feature] < best_thresh
        right_mask = ~left_mask
        random_subset_of_features1=self.random_feature(X.shape[1],2)
        random_subset_of_features2=self.random_feature(X.shape[1],2)
        left_subtree = self.build_tree(X[left_mask], y[left_mask], random_subset_of_features1,depth=depth+1)
        right_subtree = self.build_tree(X[right_mask], y[right_mask], random_subset_of_features2,depth=depth+1)
        return {
            'is_leaf': False,
            'feature': best_feature,
            'threshold': best_thresh,
            'left': left_subtree,
            'right': right_subtree
        }
    def bootstrap_data(self):
        indices = np.random.choice(len(self.x_data), len(self.x_data), replace=True)
        return self.x_data[indices], self.y_data[indices]

    #def launch_algorithm(self):
    #    result=[]
    #    for i in range(self.numb_of_tree):
    #        featur_indice=self.random_feature(self.x_data.shape[1],2)
    #        result1=self.build_tree(self.x_data,self.y_data,feature_indices=featur_indice)
    #        result.append(result1)
    #        print("hello")
    #    return result
    def launch_algorithm(self):
        result = []
        for i in range(self.numb_of_tree):
            X_sample, y_sample = self.bootstrap_data()
            feature_indices = self.random_feature(self.x_data.shape[1], 2)
            result1 = self.build_tree(X_sample, y_sample, feature_indices=feature_indices)
            result.append(result1)
            print(f"Tree {i+1} готове")
        return result
    def predict_forest(self, x, forest):
        predictions = []
        for tree in forest:
            pred = self.predict_tree(x, tree)
            predictions.append(pred)
        # голосування
        values, counts = np.unique(predictions, return_counts=True)
        majority_vote = values[np.argmax(counts)]
        return majority_vote



## Classification Task 1: Inferring Sphere Density from Chord Distributions

We aim to classify the density of spheres based on discretized chord length distributions obtained from horizontal cuts.

- **Classes**: `distribution_11` (low density) vs. `distribution_17` (high density)
- **Input**: a 10×1 feature vector (discretized chord lengths)
- **Goal**: predict which class (density level) the feature vector belongs to
- **Parameters**: 
  - `num_bins = 15` for finer discretization
  - Use a simple classifier (e.g., Decision Tree) to distinguish between the two distributions

This task mimics a supervised learning setting, where the model learns from labeled distributions and predicts the class of new observations.


In [127]:
num_bins = 15
num_cuts = 1000
radius = 0.08

positions = generate_spheres(11, radius)
chord_lengths_set_11 = collect_chords_for_cuts(positions, radius, num_cuts)
distribution_11 = distribution_chord_lengths(chord_lengths_set_11, num_bins)

positions = generate_spheres(17, radius)
chord_lengths_set_17 = collect_chords_for_cuts(positions, radius, num_cuts)
distribution_17 = distribution_chord_lengths(chord_lengths_set_17, num_bins)

X, Y = prepare_numpy_dataset([11, 17], radius, num_cuts, num_bins)

print(Y)


[11. 11. 11. ... 17. 17. 17.]


In [128]:
X_train, X_test, Y_train, Y_test=train_test_split(X=X,Y=Y)
rand_forest=RandomForest(x_data=X_train,y_data=Y_train,numb_of_tree=3,depth=4)
forest=rand_forest.launch_algorithm()

def evaluate_accuracy(X_test, Y_test, forest, model):
    correct = 0
    for x, y_true in zip(X_test, Y_test):
        y_pred = model.predict_forest(x, forest)
        if y_pred == y_true:
            correct += 1
    return correct / len(Y_test)

acc=evaluate_accuracy(X_test=X_test,Y_test=Y_test,forest=forest,model=rand_forest)
print(f"точність алгоритму дорівнює = {acc}")

Tree 1 готове
Tree 2 готове
Tree 3 готове
точність алгоритму дорівнює = 0.5666666666666667


In [72]:
def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

def load_model(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)


In [74]:
save_model(forest, 'my_forest.pkl')
loaded_forest = load_model('my_forest.pkl')


In [76]:

#new_forest=RandomForest(x_data=[],y_data=[],numb_of_tree=3,depth=4)  
#y_pred = new_forest.predict_forest(x, loaded_forest)      випробування на нових даних 

## Regression Task 2: Inferring Sphere Radius from Chord Distributions

We now approach a **regression** problem: estimating the true **radius** of the spheres based on the shape of their chord length distributions.

- **Classes (Radii)**: choose two known radii, e.g. `r = 0.04` and `r = 0.08`
- **Input**: a 10×1 discretized feature vector from chord lengths
- **Output**: estimated radius of the sphere sample
- **Goal**: train a regression model to learn the mapping from discretized distributions to true radius values

This task demonstrates how statistical features derived from 2D cuts can be used to infer underlying 3D geometric properties.


In [78]:
######### for you to fill in
import numpy as np

# Функції для генерації сфер і обчислення хорд ти вже маєш (generate_spheres, collect_chords_for_cuts, distribution_chord_lengths)

# Параметри
num_cuts = 100
num_bins = 10

# Два різних радіуси
radius_small = 0.04
radius_large = 0.08

# Генеруємо позиції сфер
positions_small = generate_spheres(17, radius_small)
positions_large = generate_spheres(17, radius_large)

# Генеруємо множини хорд
chord_lengths_small = collect_chords_for_cuts(positions_small, radius_small, num_cuts)
chord_lengths_large = collect_chords_for_cuts(positions_large, radius_large, num_cuts)

# Отримуємо розподіли довжин хорд
distribution_small = distribution_chord_lengths(chord_lengths_small, num_bins)
distribution_large = distribution_chord_lengths(chord_lengths_large, num_bins)

# Формуємо датасет
X_small = np.array(distribution_small)   # Розподіли як ознаки
X_large = np.array(distribution_large)

# Створюємо цільові значення (радіуси)
Y_small = np.full((X_small.shape[0], 1), radius_small)
Y_large = np.full((X_large.shape[0], 1), radius_large)

# Об'єднуємо в єдиний датасет
X = np.vstack((X_small, X_large))
Y = np.vstack((Y_small, Y_large))

# Все готово
print("Форма X:", X.shape)
print("Форма Y:", Y.shape)


Форма X: (166, 10)
Форма Y: (166, 1)


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Розбиваємо на тренувальні і тестові дані
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Створюємо і навчаємо модель дерева для регресії
regressor = DecisionTreeRegressor(max_depth=4, random_state=42)
regressor.fit(X_train, Y_train)

# Робимо передбачення
Y_pred = regressor.predict(X_test)

# Оцінюємо якість
mse = mean_squared_error(Y_test, Y_pred)
print(f"Середньоквадратична помилка (MSE): {mse:.5f}")

# Приклад передбачення
print("Справжні радіуси:", Y_test.flatten())
print("Передбачені радіуси:", Y_pred.flatten())


Середньоквадратична помилка (MSE): 0.00004
Справжні радіуси: [0.08 0.08 0.08 0.04 0.08 0.04 0.08 0.08 0.08 0.08 0.04 0.04 0.04 0.04
 0.04 0.04 0.08 0.04 0.08 0.04 0.08 0.04 0.04 0.04 0.08 0.08 0.08 0.04
 0.04 0.08 0.04 0.08 0.08 0.08]
Передбачені радіуси: [0.08       0.08       0.08       0.04333333 0.08       0.04333333
 0.08       0.08       0.08       0.04333333 0.04333333 0.04333333
 0.04333333 0.04333333 0.04333333 0.04333333 0.08       0.04333333
 0.08       0.04333333 0.08       0.04333333 0.04333333 0.04333333
 0.08       0.08       0.08       0.04333333 0.04333333 0.08
 0.04333333 0.08       0.08       0.08      ]


### 🔧 Lifehack: Sampling a Realistic Discretized Vector to save your time

This small code snippet is a quick way to extract a **realistic discretized vector** from a generated chord length distribution. It simulates a sphere arrangement with known radius and density, collects chord lengths from horizontal cuts, and selects the **first meaningful vector** (i.e., one that contains actual data).


In [17]:
# Parameters
num_spheres = 17
radius = 0.08
num_cuts = 100
num_bins = 10

# Generate spheres and chord lengths
positions = generate_spheres(num_spheres, radius)
chord_lengths_set = collect_chords_for_cuts(positions, radius, num_cuts)
distribution = distribution_chord_lengths(chord_lengths_set, num_bins)

# Extract a single discretized vector (e.g., the first non-empty one)
sample_vector = None
for vec in distribution:
    if vec.sum() > 0:  # Skip empty cuts
        sample_vector = vec
        break

# Show the result
print("Sample discretized vector:", sample_vector)

Sample discretized vector: [0 0 1 0 0 0 0 0 0 0]
